# Rotating convection in a cylinder

In [ ]:
from cylinderConvection.cylinderConvection import cylConvection
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Examples

## 1. Quick example

Create a class instance each time you set the tank aspect ratio Γ (Γ = diameter/height), the number of azimuthal velocity wavenumbers M, radial wavenumbers K, and passive temperature wavenumbers J.  

For this example, it's sufficient to limit our search to the first 20 x 20 velocity modes and include only the first 20 temperature modes.

In [ ]:
RotConv1 = cylConvection(Γ = 4, M = 20, K = 20, J = 20)

Compute the minimum Rayleigh numbers (Ra = α ΔT g H^3/(ν κ)) for oscillatory and wall convection, given Ekman (E = ν/(2 Ω H^2)) and Prandtl (Pr = ν/κ) numbers. Print values.

In [ ]:
RotConv1.minimizeRayleigh(E = 1e-4, Pr = 0.025, printVals=True)

## 2. Looping over a range of Ekman numbers

Create a class instance each time you set Γ, M, K, J (should take ~50 seconds):

In [ ]:
RotConv1 = cylConvection(Γ = 2, M = 90, K = 90, J = 90)

Compute the minimum Rayleigh number for each Ekman number in an array of values (should take ~2 seconds):

In [ ]:
Pr = 0.025
numE = 100;
EArr = np.logspace(-8,-3,num=numE)
RaOCylArr = np.zeros(len(EArr))
RaWArr = np.zeros(len(EArr))
m0cArr = np.zeros(len(EArr))
k0cArr = np.zeros(len(EArr))

for i in range(len(EArr)):
    E = EArr[i]
    Rac, R1c, σ1c, m0c, k0c, RaWall, RWall, σWall, mWall = RotConv1.minimizeRayleigh(E = E,Pr = Pr)
    RaOCylArr[i] = Rac
    RaWArr[i] = RaWall
    m0cArr[i] = m0c
    k0cArr[i] = k0c

Plot:

In [ ]:
colors = ['#4E8FC8','#D9A900','#C50230']

E_filt = EArr[0:-21]
k0c_filt = k0cArr[0:-21]
m,b = np.polyfit(np.log(E_filt),np.log(k0c_filt),1)

f, (ax1, ax2) = plt.subplots(2, 1,figsize=(7,7))
title = f'$\\Gamma = {RotConv1.Γ:.2f},\\quad Pr = {Pr:.3f},'
title += f'\\quad M={RotConv1.M:.0f}, \\quad K={RotConv1.K:.0f}, \\quad J={RotConv1.J:.0f}$'
ax1.set_title(title)
ax1.loglog(EArr,RaWArr,'k--',label='$Ra_W$')
ax1.loglog(EArr,RaOCylArr,color=colors[0],linestyle='--',label='$Ra_O^{cyl}$')
ax1.invert_xaxis()
ax1.set_xlim(EArr[-1],EArr[0])
ax1.set_ylim(1e4,1e11)
ax1.grid()
ax1.set_xlabel('$E$')
ax1.set_ylabel('$Ra$')
ax1.legend()

ax2.loglog(EArr,m0cArr,label='$m_{0c}$',color='#915eb5')
ax2.loglog(EArr,k0cArr,label='$k_{0c}$',color='#e39600')
ax2.loglog(EArr[EArr<=E_filt[-1]],np.exp(np.log(EArr[EArr<=E_filt[-1]])*m + b),color='gray',linestyle='--',label=f'$k$ = {np.exp(b):.2f} $E^{{{m:.2f}}}$')
ax2.invert_xaxis()
ax2.set_yticks(np.array([1,5,10,50,100]),labels=['1','5','10','50','100'])
ax2.grid(color='#D3D3D3')
ax2.set_xlim(EArr[-1],EArr[0])
ax2.set_xlabel('$E$')
ax2.set_ylabel('critical wavenumber (oscillatory)')
ax2.legend(loc='upper left')
fname = f'Γ={RotConv1.Γ:.2f}-Pr={Pr:.3f}-M={RotConv1.M:.0f}-K={RotConv1.K:.0f}-J={RotConv1.J:.0f}.jpg'
#plt.savefig(fname,dpi=300)
plt.show()

# Check against values from [Zhang & Liao (2017)](https://doi.org/10.1017/9781139024853)

In [ ]:
EZL,PrZL,ΓZL,k0cZL,m0cZL,R1cZL,σ1cZL = np.loadtxt('Zhang&Liao2017Values.csv',skiprows=1,delimiter=',',unpack=True)
ΓZL = np.unique(ΓZL).item()

In [ ]:
RotConv2 = cylConvection(Γ = ΓZL, M = 25, K = 25, J = 25)

In [ ]:
R1cArr = np.zeros(len(EZL))
σ1cArr = np.zeros(len(EZL))
m0cArr = np.zeros(len(EZL))
k0cArr = np.zeros(len(EZL))

for i in range(len(EZL)):
    E = EZL[i]
    Pr = PrZL[i]
    Rac, R1c, σ1c, m0c, k0c, RaWall, RWall, σWall, mWall = RotConv2.minimizeRayleigh(E = E,Pr = Pr)
    R1cArr[i] = R1c
    σ1cArr[i] = σ1c
    m0cArr[i] = m0c
    k0cArr[i] = k0c

numWrongR1c = np.sum(np.abs((R1cZL - R1cArr)/R1cZL) > 1e-3)
numWrongσ1c = np.sum(np.abs((σ1cZL - σ1cArr)/σ1cZL) > 1e-3)
numWrongm0c = np.sum(np.abs(m0cArr-m0cZL))
numWrongk0c = np.sum(np.abs(k0cArr-k0cZL))

if (numWrongR1c+numWrongσ1c+numWrongm0c+numWrongk0c) == 0:
    msg = f'Computed values are within 0.1% of published values (at'
    msg+= f' M = {RotConv2.M:.0f}, K = {RotConv2.K:.0f}, and J = {RotConv2.J:.0f})'
    print(msg)
else:
    print("Something's wrong.")

# Flow field

Initialize class for a tank with Γ = 2. Find the onset oscillatory and bulk modes.

In [ ]:
RotConv3 = cylConvection(Γ = 2, M = 30, K = 30, J = 30)
RotConv3.minimizeRayleigh(E = 1e-06, Pr = 0.025, printVals=True)

Get velocity meshgrids of the onset oscillatory mode. Should take ~16 s:

In [ ]:
gdata = RotConv3.gridData(ns=51,nφ=51,nz=21,t=np.linspace(0,20,50))

Generate synthetic ultrasonic Doppler velocimetric (UDV) data along a chord of the cylinder. Provide the transducer position and beam path endpoint in cylindrical coordinates.

In [ ]:
s1=0.5*RotConv3.Γ
φ1=np.pi/4
z1=0.75
s2=0.5*RotConv3.Γ
φ2=3*np.pi/4
z2=0.75
t=np.linspace(0,75,50)
gates=50

gdataDop, beamPath = RotConv3.synthDoppler(s1, φ1,z1,s2,φ2,z2,t,gates)

Plot:

In [ ]:
zind=15
tind=0
z=gdata['z'][0,0,zind,0]
t=gdata['t'][0,0,0,tind]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
ax1.plot(beamPath['x'],beamPath['y'],'k',label='UDV beam path')
ax1.arrow(beamPath['x'][int(gates/2)],beamPath['y'][int(gates/2)],
          beamPath['x'][int(gates/2)+1]-beamPath['x'][int(gates/2)],
          beamPath['y'][int(gates/2)+1]-beamPath['y'][int(gates/2)],
          zorder=2, head_width=0.05, color='k')
ax1.legend()
ax1.set_xlabel('$x/H$',fontsize=15)
ax1.set_ylabel('$y/H$',fontsize=15)
ax1.set_title(f'$t \Omega$={t:.1f}')
im1 = ax1.contourf(gdata['x'][...,zind,tind],gdata['y'][...,zind,tind],gdata['uphi'][...,zind,tind],levels=20,cmap='RdBu')
plt.colorbar(im1,ax=ax1).set_label(label='$u_\phi$',size=15)
ax1.set_aspect('equal', 'box')

im2 = ax2.contourf(gdataDop['t'], gdataDop['l'], gdataDop['uDop'],levels=20,cmap='RdBu')
plt.colorbar(im2,ax=ax2).set_label(label='$u_{DOP}$',size=15)
ax2.set_xlabel('$t \Omega$',fontsize=15)
ax2.set_ylabel('$d$',fontsize=15)
ax2.set_title('Hovmoller plot of synthetic UDV data')
title = f'Γ={RotConv3.Γ:.1f}, E={RotConv3.E:.1e}, Pr={RotConv3.Pr:.1e}'
title += f', z={z:.2f}'
plt.suptitle(title,fontsize=15)
plt.subplots_adjust(wspace=0.15)
plt.tight_layout()
#plt.savefig('synthUDV.jpg',dpi=300)